In [1]:
import numpy as np
import faiss
import os
import jieba
import copy
from FlagEmbedding import FlagModel
import json

/home/lsh/anaconda3/envs/DF-retrieval/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!curl -I https://huggingface.co

HTTP/1.1 200 Connection established

HTTP/2 200 
content-type: text/html; charset=utf-8
content-length: 111881
date: Fri, 15 Nov 2024 13:17:31 GMT
x-powered-by: huggingface-moon
cross-origin-opener-policy: same-origin
referrer-policy: strict-origin-when-cross-origin
x-request-id: Root=1-673749eb-5407be3d778c4fd212bb7aaa
x-frame-options: DENY
etag: W/"1b509-0p4fEsoi6Qi8sxAneNX9zpi2XEM"
vary: Accept-Encoding
x-cache: Hit from cloudfront
via: 1.1 fb23da0134ded13322b39ad952ac5222.cloudfront.net (CloudFront)
x-amz-cf-pop: HKG1-P1
x-amz-cf-id: cY0FH3QEUCUm9w1s6QgQY9rR-5gFaR_PXaoDIoppkMG-9eOaGF4SmA==
age: 46



In [ ]:
# %env ALL_PROXY=http://localhost:7890
# %env HTTP_PROXY=http://localhost:7890
# %env HTTPS_PROXY=http://localhost:7890

In [3]:

!curl -I https://huggingface.co
model = FlagModel('BAAI/bge-large-zh-v1.5',
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                  use_fp16=True)

HTTP/1.1 200 Connection established

HTTP/2 200 
content-type: text/html; charset=utf-8
content-length: 111881
date: Fri, 15 Nov 2024 13:17:31 GMT
x-powered-by: huggingface-moon
cross-origin-opener-policy: same-origin
referrer-policy: strict-origin-when-cross-origin
x-request-id: Root=1-673749eb-5407be3d778c4fd212bb7aaa
x-frame-options: DENY
etag: W/"1b509-0p4fEsoi6Qi8sxAneNX9zpi2XEM"
vary: Accept-Encoding
x-cache: Hit from cloudfront
via: 1.1 2c8849b76728bafd760d0b71186f7bd2.cloudfront.net (CloudFront)
x-amz-cf-pop: HKG1-P1
x-amz-cf-id: HiCc64rXkckgsqMFhTlzpjVa6kN0P758cDrw4mxskWBVi5P_6AOWWQ==
age: 50



In [4]:
import torch
torch.__version__
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.4


In [5]:
# with open('resources/train-v2.0.json', 'r') as file:
#     squad_data = json.load(file)

folder_path = '/home/lsh/DM-DataFountain-Retrieval/resources/temp'
output_path = '/home/lsh/DM-DataFountain-Retrieval/resources'


In [15]:

# 遍历文件夹及其子文件夹中的所有文件
vectors = []
datas = {'file_path': [], 'vectors': [], 'text_num': []}

file_num = 1
text_num = 1

for item in os.listdir(folder_path):
    file_path = os.path.join(folder_path, item)
    if not os.path.isfile(file_path):
        continue
    with open(file_path, 'r', encoding='utf-8') as f:
        text_num = 0
        data = json.load(f)
        print(file_path)
        for text in data:
            if isinstance(text, str):
                vec = model.encode(text)
                vectors.append(vec)
                datas['file_path'].append(file_path)
                datas['text_num'].append(text_num)
                datas['vectors'].append(vec.tolist())
            else:
                print('wrong:', end='')
                print(text)
            text_num += 1
        print('file:' + str(file_num) + 'complete')
        file_num += 1
            
with open(output_path + '/vectors.json', 'w') as file:
    json.dump(datas, file, ensure_ascii=False, indent=4)

/home/lsh/DM-DataFountain-Retrieval/resources/temp/146.json
file:1complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/168.json
file:2complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/181.json
file:3complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/53.json
file:4complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/70.json
file:5complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/156.json
file:6complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/81.json
file:7complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/66.json
file:8complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/12.json
file:9complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/9.json
file:10complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/8.json
file:11complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/79.json
file:12complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/126.json
file:13complete
/home/lsh/DM-DataFountain-Retri

In [16]:
print(len(vectors))

5284


In [8]:

with open(output_path + '/vectors.json', 'r') as file:
    datas = json.load(file)

In [17]:
def convert_text_to_vector(text):
    vec = model.encode(text)
    return vec

paragraph_vectors = np.stack(vectors).astype('float32')
dimension = paragraph_vectors.shape[1]
indexx = faiss.IndexFlatL2(dimension)
indexx.add(paragraph_vectors)

In [10]:
print(dimension)

1024


<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7e1c25f944b0> >


In [23]:
def search_for_paragraphs(search_term : str, num_results, index) -> str:
    print('search_term:' + str(type(search_term)))
    block1 = str(search_term).split("，")
    block = []
    for ttt in block1:
        for tttt in ttt.split('的'):
            block.append(tttt)
    block.reverse()
    print(block)
    temp_index = index
    temp_datas = datas
    max_num = 1000
    # for b in block:
    #   temp_index, temp_datas = search_for_block(b, max_num, temp_index, temp_datas)
    #   max_num = max(int(max_num * 0.5), 10)
    search_vector = convert_text_to_vector(search_term)
    search_vector = np.array([search_vector]).astype('float32')
    distances, indexes = temp_index.search(search_vector, num_results)
    anss = ''
    for i, (distance, index) in enumerate(zip(distances[0], indexes[0])):
        path = temp_datas['file_path'][index]
        num = temp_datas['text_num'][index]
        with open(path, 'r') as file:
            data = json.load(file)
            anss = anss + '\n' + data[num]
    return anss

def search_for_block(b, search_num ,index, tdatas):
    search_vector = convert_text_to_vector(b)
    search_vector = np.array([search_vector]).astype('float32')
    print(search_num)
    distances, indexes = index.search(search_vector, search_num)
    print(indexes)
    temp_vectors = []
    temp_datas = {'file_path': [], 'vectors': [], 'text_num': []}
    for i, (distance, index) in enumerate(zip(distances[0], indexes[0])):
        vector = tdatas['vectors'][index]
        temp_vectors.append(np.array(vector))
        temp_datas['file_path'].append(tdatas['file_path'][index])
        temp_datas['vectors'].append(tdatas['vectors'][index])
        temp_datas['text_num'].append(tdatas['text_num'][index])
    block_vectors = np.stack(temp_vectors).astype('float32')
    temp_dimension = block_vectors.shape[1]
    temp_index = faiss.IndexFlatL2(temp_dimension)
    temp_index.add(block_vectors)
    return temp_index, temp_datas


In [24]:
ans = search_for_paragraphs('帮我查一下，2023年联通5G中频基站的数量和新增量分别是多少？', 3, indexx)
print(ans)

search_term:<class 'str'>
['数量和新增量分别是多少？', '2023年联通5G中频基站', '帮我查一下']

2023年8月，中国电信获批重耕800MHz频段资源用于5G网络，积极开展约25万个800MHz5G基站集采工作4。截至人口数按141175万计算https://news.youth.cn/gn/202304/t20230428_14488190.htm，工信部：5G网络已覆盖全国所有地级市、县城https://www.sohu.com/a/606250739_128075，中国联通：本年度部署17万个900M低频基站https://www.c114.com.cn/news/117/a1244028.html，中国电信启动800M5G基站集采：预估25万站中国5G发展和经济社会影响白皮书（2023年）2023年6月，中国移动和中国广电共完成57.8万700MHz5G基站建设5，进一步提升农村及偏远地区网络覆盖。5G行业虚拟专网形成规模发展态势。
其中，2023年1-10月我国新增5G基站中国5G发展和经济社会影响白皮书（2023年）达90.3万个，超额完成60万的年新增目标；每万人基站数达22.8个1，已达到《“十四五”信息通信行业发展规划》目标的87.7%。来源：工业和信息化部2我国5G基站建设数量（单位：万）共建共享持续深化。我国5G共建共享基站超150万个2，推动构建高效绿色、建设集约的低碳通信网络。
5G中频基站新增万站，4G共享基站超过站。与友商在新疆启动全球首个异网漫游试商用，进一步提升边远地区网络资源使用效益；努力推动传输线路、管道光缆等基础设施的共建共享共维取得积极成效。以实际行动响应“双碳”政策，2023年节省运营开支约人民币390亿元，减少碳排放约1,150万吨，累计节省资本开支约人民币3,400亿元。


In [25]:
import pandas as pd
question_path = '/home/lsh/DM-DataFountain-Retrieval/resources/B_question.csv'
questions = pd.read_csv(question_path)
print(questions['question'])

0      根据2023年年度报告摘要，中国联通在中央企业品牌建设能力对标中位于通信行业第几名？
1                帮我查一下，2023年联通5G中频基站的数量和新增量分别是多少？
2     查询2023年年度报告摘要，告诉我联通算网数智业务发展能力的增强主要体现在哪几个方面。
3                        2023年中国联通通信业务的用户规模达到了多少？
4                        2023年前三季度中国联通移动用户规模情况如何？
                         ...                     
75                          2024年青海联通的网络建设覆盖情况如何？
76          中国联通成为北京2022年冬奥会和冬残奥会官方通信服务合作伙伴是在哪一天？
77                  截至2024年7月19日，中国联通全网智算算力达到了多少？
78                     中国联通在智慧城市和数字乡村平台建设方面有哪些应用？
79                      中国联通累计开通的4G和5G共享基站分别有多少站？
Name: question, Length: 80, dtype: object


In [26]:
ans = {'ques_id': [], 'question': [], 'answer': [], 'embedding': []}
for index, row in questions.iterrows():
    ans['ques_id'].append(row['ques_id'])
    ans['question'].append(row['question'])
    temp = search_for_paragraphs(row['question'], 3, indexx)
    ans['answer'].append(temp)
    emb = model.encode(temp)
    emb = [str(x) for x in emb.tolist()]
    ans['embedding'].append(','.join(emb))
    

search_term:<class 'str'>
['中国联通在中央企业品牌建设能力对标中位于通信行业第几名？', '根据2023年年度报告摘要']
search_term:<class 'str'>
['数量和新增量分别是多少？', '2023年联通5G中频基站', '帮我查一下']
search_term:<class 'str'>
['增强主要体现在哪几个方面。', '告诉我联通算网数智业务发展能力', '查询2023年年度报告摘要']
search_term:<class 'str'>
['用户规模达到了多少？', '2023年中国联通通信业务']
search_term:<class 'str'>
['2023年前三季度中国联通移动用户规模情况如何？']
search_term:<class 'str'>
['2023年前三季度中国联通新获专利数量同比提升了多少？']
search_term:<class 'str'>
['开展情况。', '告诉我2023年第一季度联通“大计算”业务']
search_term:<class 'str'>
['2023年第一季度中国联通大数据收入较去年同期提升了多少？']
search_term:<class 'str'>
['企业转型升级？', '公司正在从传统管道运营商向什么类型', '中国联通2023年第一季度报告指出']
search_term:<class 'str'>
['告诉我中国联通2023年年度股东大会现场会议于何时召开？']
search_term:<class 'str'>
['联通在原创技术策源地建设上取得了哪些成果？', '2023年度董事会报告中指出']
search_term:<class 'str'>
['中国联通监事会召开了几次会议？', '在2023年度']
search_term:<class 'str'>
['情况。', '告诉我2021年联通大数据业务收入和市场份额', '查一下年度报告']
search_term:<class 'str'>
['增长百分比是多少？', '净利润相比上年', '中国联通2021年归属于上市公司股东']
search_term:<class 'str'>
['2021年上半年联通履行了哪些社会责任？']
search_term:<class 'st

In [27]:
df = pd.DataFrame(ans)

output_file_path = '/home/lsh/DM-DataFountain-Retrieval/resources/submit.csv'  # 指定输出文件的路径和文件名
df.to_csv(output_file_path, index=False)  # index=False表示不将行索引写入文件